# Parallel computing and reusing realizations

In this example, we demonstrate two features of the library that help reduce calculation time:
* Parallel computing – a method of performing multiple calculations simultaneously to improve efficiency.
* Reusing realizations – avoiding redundant computations by utilizing previously generated realizations.

We apply these features to calculate the probability of levee failure due to wave overtopping.

First, we import the necessary packages:

In [2]:
from probabilistic_library import ReliabilityProject, DistributionType, ReliabilityMethod
import time

We consider the limit state function for wave overtopping (which we have artificially slowed down):

In [3]:
from utils.models import z_func_overtopping

class ZFunctionOvertopping():

    def __init__(self):
        pass
    def z_sleep(self, h, hm0, tm10, wave_direction, dike_normal, y_crest, q_crit):

        time.sleep(0.1)
        Z = z_func_overtopping(h, hm0, tm10, wave_direction, dike_normal, y_crest, q_crit)

        return Z

And the following reliability project, in which we apply the `crude_monte_carlo` method:

In [ ]:
def define_project():

    project = ReliabilityProject()
    project.model = ZFunctionOvertopping.z_sleep
    
    project.variables["h"].distribution = DistributionType.log_normal
    project.variables["h"].mean = 1.5
    project.variables["h"].deviation = 0.05

    project.variables["hm0"].distribution = DistributionType.log_normal
    project.variables["hm0"].mean = 1.5
    project.variables["hm0"].deviation = 0.25

    project.variables["tm10"].distribution = DistributionType.log_normal
    project.variables["tm10"].mean = 3
    project.variables["tm10"].deviation = 0.5

    project.variables["wave_direction"].distribution = DistributionType.deterministic
    project.variables["wave_direction"].mean = 0.0

    project.variables["dike_normal"].distribution = DistributionType.deterministic
    project.variables["dike_normal"].mean = 0.0

    project.variables["y_crest"].distribution = DistributionType.deterministic
    project.variables["y_crest"].mean = 6.0

    project.variables["q_crit"].distribution = DistributionType.log_normal
    project.variables["q_crit"].mean = 0.001
    project.variables["q_crit"].deviation = 0.01

    project.settings.reliability_method = ReliabilityMethod.crude_monte_carlo
    project.settings.minimum_samples = 1000
    project.settings.maximum_samples = 1000
    project.settings.variation_coefficient = 0.02

    return project

### Parallel computing

If not specified, the calculations are performed on a single processor. To utilize multiple processors, we adjust the setting `project.settings.max_parallel_processes`. 

With the following code, we analyze the effect of parallel computing on the calculation time:

In [ ]:
max_parallel_processes = [1, 4, 8]

for val in max_parallel_processes:

    project = define_project()
    project.settings.max_parallel_processes = val

    t = time.time()
    project.run()
    elapsed = time.time() - t

    print(f"Max parallel processes: {val}")
    print(f"Reliability index: {project.design_point.reliability_index}")
    print(f"Time elapsed: {elapsed} seconds")

Max parallel processes: 1
Reliability index: 1.3407550336902165
Time elapsed: 10.172954797744751 seconds
----------------------------------------------------------------


### Reusing realizations

Another useful feature of the library is the ability to reuse realizations - this is possible if the project is not redefined when it is run again. This is demonstrated in the following example: realizations from a previous run with $8$ processors are reused in subsequent calculations, significantly reducing computation time.

This functionality is particularly valuable in sensitivity analyses (if multiple model outputs are required). It is also beneficial when applying the Crude Monte Carlo method in reliability analysis. For instance, if a calculation is first performed with $10,000$ samples and later extended to $20,000$ samples, the initial $10,000$ samples are reused in the second run, avoiding unnecessary recomputation.

In [ ]:
for id in range(5):
    
    t = time.time()
    project.run()
    elapsed = time.time() - t

    print(f"Run number: {id}")
    print(f"Reliability index: {project.design_point.reliability_index}")
    print(f"Time elapsed: {elapsed} seconds")

Run number: 0
Reliability index: 1.3407550336902165
Time elapsed: 0.001001119613647461 seconds
----------------------------------------------------------------
Run number: 1
Reliability index: 1.3407550336902165
Time elapsed: 0.0 seconds
----------------------------------------------------------------
Run number: 2
Reliability index: 1.3407550336902165
Time elapsed: 0.0005528926849365234 seconds
----------------------------------------------------------------
Run number: 3
Reliability index: 1.3407550336902165
Time elapsed: 0.0 seconds
----------------------------------------------------------------
Run number: 4
Reliability index: 1.3407550336902165
Time elapsed: 0.0010864734649658203 seconds
----------------------------------------------------------------
